In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>, Cristovao da Cruz e Silva
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [1]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil
from numpy import savetxt

os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)


from scripts.log_action import log_action_v2

In [2]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

<Figure size 1000x1000 with 0 Axes>

# Set defaults

In [29]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_Bare_2", "ET2_Bare_3"]
chip_fignames = ["ETROC 2.01 Bare Board 2", "ETROC 2.01 Bare Board 3"]

#chip_names = ["ET2_Bare_4"]
#chip_fignames = ["ETROC 2.01 Bare Board 4"]

chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60, 0x61]
#chip_addresses = [0x60]
ws_addresses = [None, None]

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

config_outdir = Path('../ETROC-Data')
config_outdir = config_outdir / "ChipConfig"
config_outdir.mkdir(exist_ok=True)

log_path = Path(f'/home/{os.getlogin()}/ETROC2/i2c_gui')

full_col_list, full_row_list = np.meshgrid(np.arange(16),np.arange(16))
full_scan_list = list(zip(full_row_list.flatten(),full_col_list.flatten()))

# Make i2c_connection class object

In [30]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,[("1","1"),("1","1")])

# Config chips

## Key is (Disable Pixels, Auto Cal, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [31]:
# (WS Prep Pixel and Peri) - 0 - (disable & auto_cal all pixels) - (auto_TH_CAL) - (disable default all pixels) - (set basic peripherals) - (peripheral reg check) -  (pixel ID check)
log_action_v2(log_path, "DAQ", "Config", "Initial")
i2c_conn.config_chips('00001101')
for address in chip_addresses:
    log_action_v2(log_path, "DAQ", "Config", "High Power")
    i2c_conn.set_power_mode_scan_list(address, full_scan_list, 'high')
log_action_v2(log_path, "DAQ", "Config", "Baseline")
i2c_conn.config_chips('00010000')
log_action_v2(log_path, "DAQ", "Config", "Done")

Passed pixel check for chip: 0x60
Peripherals set for chip: 0x60
Disabled pixels for chip: 0x60
Passed pixel check for chip: 0x61
Peripherals set for chip: 0x61
Disabled pixels for chip: 0x61


Calibrating row: 100%|██████████| 16/16 [01:31<00:00,  5.70s/it]


Auto calibration finished for chip: 0x60


Calibrating row: 100%|██████████| 16/16 [01:31<00:00,  5.70s/it]

Auto calibration finished for chip: 0x61


## Dump I2C Config


In [32]:
i2c_conn.dump_config(config_outdir, "FirstSetup_Feb13_SEUEquivalence")

2024-02-13 20:20:48,354 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set
2024-02-13 20:22:28,537 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set


## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [33]:
def make_baselines(do_config = True, power_mode = 'high', calibrate_chip=True, calibrate_pixels=None):
    if do_config:
        log_action_v2(log_path, "DAQ", "Config", "Basic and Disable")
        i2c_conn.config_chips('00001100')
        for address in chip_addresses:
            if power_mode == 'low':
                log_action_v2(log_path, "DAQ", "Config", "Low Power")
            elif power_mode == 'high':
                log_action_v2(log_path, "DAQ", "Config", "High Power")
            else:
                log_action_v2(log_path, "DAQ", "Config", "Set Power")
            i2c_conn.set_power_mode_scan_list(address, full_scan_list, power_mode = power_mode)

    histdir = Path('/media/daq/T7/ETROC-History-SEUEquivalence')
    histdir.mkdir(exist_ok=True)
    histfile = histdir / 'BaselineHistory.sqlite'

    log_action_v2(log_path, "DAQ", "Config", "Baseline")
    if(calibrate_chip): i2c_conn.config_chips('00010000')
    elif(calibrate_pixels is not None):
        data = []
        for aidx,address in enumerate(chip_addresses):
            for row,col in calibrate_pixels:
                i2c_conn.auto_cal_pixel(chip_name=chip_names[aidx], row=row, col=col, verbose=True, chip_address=address, data=data)
        BL_df = pandas.DataFrame(data = data)
        with sqlite3.connect(histfile) as sqlconn:
            BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)
        del data, BL_df
        return
    
    log_action_v2(log_path, "DAQ", "Config", "Done")

    # from mpl_toolkits.axes_grid1 import make_axes_locatable
    for chip_address, chip_figname, chip_figtitle in zip(chip_addresses,chip_fignames,chip_fignames):
        BL_map_THCal,NW_map_THCal,BL_df,_ = i2c_conn.get_auto_cal_maps(chip_address)
        fig = plt.figure(dpi=200, figsize=(20,10))
        gs = fig.add_gridspec(1,2)

        ax0 = fig.add_subplot(gs[0,0])
        ax0.set_title(f"{chip_figtitle}: BL (DAC LSB)", size=17, loc="right")
        img0 = ax0.imshow(BL_map_THCal, interpolation='none')
        ax0.set_aspect("equal")
        ax0.invert_xaxis()
        ax0.invert_yaxis()
        plt.xticks(range(16), range(16), rotation="vertical")
        plt.yticks(range(16), range(16))
        hep.cms.text(loc=0, ax=ax0, fontsize=17, text="Preliminary")
        divider = make_axes_locatable(ax0)
        cax = divider.append_axes('right', size="5%", pad=0.05)
        fig.colorbar(img0, cax=cax, orientation="vertical")

        ax1 = fig.add_subplot(gs[0,1])
        ax1.set_title(f"{chip_figtitle}: NW (DAC LSB)", size=17, loc="right")
        img1 = ax1.imshow(NW_map_THCal, interpolation='none')
        ax1.set_aspect("equal")
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        plt.xticks(range(16), range(16), rotation="vertical")
        plt.yticks(range(16), range(16))
        hep.cms.text(loc=0, ax=ax1, fontsize=17, text="Preliminary")
        divider = make_axes_locatable(ax1)
        cax = divider.append_axes('right', size="5%", pad=0.05)
        fig.colorbar(img1, cax=cax, orientation="vertical")

        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

        for x in range(16):
            for y in range(16):
                ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
                ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
        plt.tight_layout()
        plt.savefig(fig_path+"/BL_NW_"+chip_figname+"_"+timestamp+".png")
        plt.show()

        with sqlite3.connect(histfile) as sqlconn:
            BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)
        # savetxt(histdir / f'{chip_figname}_BL_{timestamp}.csv', BL_map_THCal, delimiter=',')
        # savetxt(histdir / f'{chip_figname}_NW_{timestamp}.csv', NW_map_THCal, delimiter=',')

    time.sleep(1)

In [34]:
make_baselines(True)

Peripherals set for chip: 0x60
Disabled pixels for chip: 0x60
Peripherals set for chip: 0x61
Disabled pixels for chip: 0x61


Calibrating row: 100%|██████████| 16/16 [02:41<00:00, 10.08s/it]


Auto calibration finished for chip: 0x60


Calibrating row: 100%|██████████| 16/16 [02:39<00:00,  9.95s/it]


Auto calibration finished for chip: 0x61


# Define Utility Functions

In [35]:
def do_pixel_operations(scan_list, board_offsets=None, noisy_pixels=None, power_mode='high'):
    log_action_v2(log_path, "DAQ", "Config", "Pixel Operations")
    i2c_conn.enable_select_pixels_in_chips(scan_list,QInjEn=True,Bypass_THCal=False,specified_addresses=chip_addresses[:], power_mode=power_mode)

    # offset = 0x0f
    # for chip_address in chip_addresses[:]:
    #     chip = i2c_conn.get_chip_i2c_connection(chip_address)
    #     row_indexer_handle,_,_ = chip.get_indexer("row")
    #     column_indexer_handle,_,_ = chip.get_indexer("column")
    #     for row, col in scan_list:
    #         print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
    #         column_indexer_handle.set(col)
    #         row_indexer_handle.set(row)
    #         i2c_conn.pixel_decoded_register_write("TH_offset", format(offset, '06b'), chip)
    #     del chip, row_indexer_handle, column_indexer_handle

    if board_offsets is not None:
        time.sleep(1)

        for chip_address in chip_addresses[:]:
            if chip_address not in board_offsets:
                continue

            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            row_indexer_handle,_,_ = chip.get_indexer("row")
            column_indexer_handle,_,_ = chip.get_indexer("column")
            for row, col in scan_list:
                print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col}) Setting Offset to {board_offsets[chip_address]}")
                column_indexer_handle.set(col)
                row_indexer_handle.set(row)
                i2c_conn.pixel_decoded_register_write("QSel", format(0x1e, '05b'), chip)
                i2c_conn.pixel_decoded_register_write("TH_offset", format(board_offsets[chip_address], '06b'), chip)
            del chip, row_indexer_handle, column_indexer_handle

    if noisy_pixels is not None:
        time.sleep(1)

        for chip_address in chip_addresses:
            if chip_address not in noisy_pixels:
                continue
            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            row_indexer_handle,_,_ = chip.get_indexer("row")
            column_indexer_handle,_,_ = chip.get_indexer("column")
            for row, col in noisy_pixels[chip_address]:
                print(f"Masking from trigger of chip {hex(chip_address)} Pixel ({row},{col})")
                column_indexer_handle.set(col)
                row_indexer_handle.set(row)

                i2c_conn.pixel_decoded_register_write("disTrigPath", "1", chip)
                #i2c_conn.pixel_decoded_register_write("DAC", format(1023, '010b'), chip)
                #i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", chip)

    time.sleep(1)

In [36]:
self_trigger_delay = 484
#LED_pages = ["0x0000","0x0004","0x0008","0x000C"] # LED pages to cycle through
LED_pages = ["0x0000", "0x0004"] # LED pages to cycle through
active_channels = "0x0003"
polarity = "0x0027" # or "0x0023"
hostname = "192.168.2.3"

In [37]:
def run_daq(trigger_board, trigger_board_name, run_number, total_time, daq_time, run_options="--compressed_binary --skip_translation -v", allow_overwrite = False, reset_string = "--check_valid_data_start --clear_fifo"):
    out_dir = Path('/media/daq/T7/ETROC-Data')
    if not out_dir.exists():
        raise RuntimeError("The base output directory does not exist... there may be a script config problem")
    if not allow_overwrite:
        run_dir = out_dir / f"Run_{run_number}"
        if run_dir.exists():
            raise RuntimeError("Run directory already exists, I am not going to overwrite")

    trigger_bit_delay = int(trigger_board+'11'+format(self_trigger_delay, '010b'), base=2)
    trigger_bit_delay = 0x3800
    process_time = daq_time + 5
    iterations = ceil(total_time/process_time)
    LED_id = 0

    for it in range(iterations):
        LED_page = LED_pages[LED_id]
        LED_id += 1
        if LED_id >= len(LED_pages): LED_id = 0
        reset_string = "--check_valid_data_start"
        if(it==0): reset_string = reset_string
        parser = parser_arguments.create_parser()
        (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t {process_time} --run_name Run_{run_number} -o loop_{it} -w -s {LED_page} -p {polarity} -d {trigger_bit_delay} -a {active_channels} {run_options} --ssd --start_DAQ_pulse --stop_DAQ_pulse {reset_string}".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Run_{run_number}_loop_{it}'))
        process.start()

        IPC_queue.put('memoFC Start Triggerbit BCR L1A QInj')
        while not IPC_queue.empty():
            pass
        time.sleep(daq_time)
        IPC_queue.put('stop DAQ')
        IPC_queue.put('memoFC Triggerbit')
        while not IPC_queue.empty():
            pass
        IPC_queue.put('allow threads to exit')
        process.join()

        del IPC_queue, process, parser

# Run DAQ

In [38]:
trigger_board = '0001'
trigger_board_name = chip_names[0]

col_list = [8, 2, 8, 2]
row_list = [0, 0, 2, 2]
# col_list = [8, 2, 8]
# row_list = [0, 0, 2]
# col_list = [8, 2]
# row_list = [0, 0]
# col_list = [8]
# row_list = [0]
#col_list = []
#row_list = []
scan_list = list(zip(row_list, col_list))

board_offsets = {
    0x60: 0x14,
    0x61: 0x14,
    #0x62: 0x0f,
    #0x64: 0x0a,
}
noisy_pixels = {
    #0x60: [ # (row, col)
    #    (10, 8),
    #    (13, 4),
    #],
}
power_mode = 'high'

## Pixel Operations

In [39]:
make_baselines(do_config = True, power_mode = power_mode, calibrate_chip=False, calibrate_pixels=scan_list)
do_pixel_operations(scan_list, board_offsets = board_offsets, noisy_pixels = noisy_pixels, power_mode = power_mode)

Peripherals set for chip: 0x60
Disabled pixels for chip: 0x60
Peripherals set for chip: 0x61
Disabled pixels for chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x61
Enabled pixel (0,8) for chip: 0x60
Enabled pixel (0,2) for chip: 0x60
Enabled pixel (2,8) for chip: 0x60
Enabled pixel (2,2) for chip: 0x60
Enabled pixel (0,8) for chip: 0x61
Enabled pixel (0,2) for chip: 0x61
Enabled pixel (2,8) for chip: 0x61
Enabled pixel (2,2) for chip: 0x61
Operating on chip 0x60 Pixel (0,8) Setting O

# Calibrate FC for all I2C

In [40]:
time.sleep(2)
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)
time.sleep(2)
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

Reset Global Readout done for chip: 0x60
asyAlignFastcommand action done for chip: 0x60
Reset Global Readout done for chip: 0x61
asyAlignFastcommand action done for chip: 0x61
Reset Global Readout done for chip: 0x60
asyAlignFastcommand action done for chip: 0x60
Reset Global Readout done for chip: 0x61
asyAlignFastcommand action done for chip: 0x61


# Calibrate PLL

In [20]:
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)

PLL Calibrated for chip: 0x60
PLL Calibrated for chip: 0x61


## Short Run for testing

In [41]:
# run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number="Test_short_Jan27_1", total_time=1, daq_time=30, allow_overwrite=True, run_options="--compressed_binary -v", reset_string="--check_valid_data_start --clear_fifo")
log_action_v2(log_path, "DAQ", "Run", "Test Start")
run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number="Test_short_Feb13_12", total_time=1, daq_time=10, allow_overwrite=True, run_options="--compressed_binary -v", reset_string="--check_valid_data_start --clear_fifo")
log_action_v2(log_path, "DAQ", "Run", "Test Stop")

## SEU Equivalence
(only for running long tests outside of SEU)

In [42]:
total_runs = 40
each_run_time = 1
each_daq_time = 60*3  # Average of 3 minutes per run from Jan24

for run_no in range(total_runs):
    make_baselines(do_config = True, power_mode = power_mode, calibrate_chip=False, calibrate_pixels=scan_list)
    do_pixel_operations(scan_list, board_offsets = board_offsets, noisy_pixels = noisy_pixels, power_mode = power_mode)

    time.sleep(40)

    i2c_conn.dump_config(config_outdir, "PreRun")

    time.sleep(20)

    run_name = f"SEUEquivalenceFeb13_{run_no}"

    log_action_v2(log_path, "DAQ", "Run", "Start")
    print(f"Run {run_name} started at:", datetime.datetime.now().isoformat(sep=" "))
    run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number=run_name, total_time=each_run_time, daq_time=each_daq_time, run_options="--compressed_binary --skip_translation", reset_string="--check_valid_data_start")
    print(f"Run {run_name} ended at:", datetime.datetime.now().isoformat(sep=" "))
    log_action_v2(log_path, "DAQ", "Run", "Start")

    i2c_conn.dump_config(config_outdir, "PostRun")

    time.sleep(10)

Peripherals set for chip: 0x60
Disabled pixels for chip: 0x60
Peripherals set for chip: 0x61
Disabled pixels for chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x61
Enabled pixel (0,8) for chip: 0x60
Enabled pixel (0,2) for chip: 0x60
Enabled pixel (2,8) for chip: 0x60
Enabled pixel (2,2) for chip: 0x60
Enabled pixel (0,8) for chip: 0x61
Enabled pixel (0,2) for chip: 0x61
Enabled pixel (2,8) for chip: 0x61
Enabled pixel (2,2) for chip: 0x61
Operating on chip 0x60 Pixel (0,8) Setting O

2024-02-13 20:34:36,022 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set
2024-02-13 20:36:16,252 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set


Run SEUEquivalenceFeb13_0 started at: 2024-02-13 20:36:36.317005
Run SEUEquivalenceFeb13_0 ended at: 2024-02-13 20:39:42.738137


2024-02-13 20:41:22,919 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set
2024-02-13 20:43:03,091 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set


Peripherals set for chip: 0x60
Disabled pixels for chip: 0x60
Peripherals set for chip: 0x61
Disabled pixels for chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x61
Enabled pixel (0,8) for chip: 0x60
Enabled pixel (0,2) for chip: 0x60
Enabled pixel (2,8) for chip: 0x60
Enabled pixel (2,2) for chip: 0x60
Enabled pixel (0,8) for chip: 0x61
Enabled pixel (0,2) for chip: 0x61
Enabled pixel (2,8) for chip: 0x61
Enabled pixel (2,2) for chip: 0x61
Operating on chip 0x60 Pixel (0,8) Setting O

...

2024-02-14 04:15:38,712 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set
2024-02-14 04:17:18,964 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set


Peripherals set for chip: 0x60
Disabled pixels for chip: 0x60
Peripherals set for chip: 0x61
Disabled pixels for chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x60
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (0,2) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,8) on chip: 0x61
Auto calibration done (TDC=0 + DAC=BL) for pixel (2,2) on chip: 0x61
Enabled pixel (0,8) for chip: 0x60
Enabled pixel (0,2) for chip: 0x60
Enabled pixel (2,8) for chip: 0x60
Enabled pixel (2,2) for chip: 0x60
Enabled pixel (0,8) for chip: 0x61
Enabled pixel (0,2) for chip: 0x61
Enabled pixel (2,8) for chip: 0x61
Enabled pixel (2,2) for chip: 0x61
Operating on chip 0x60 Pixel (0,8) Setting O

2024-02-14 04:20:48,410 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set
2024-02-14 04:22:28,692 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set


Run SEUEquivalenceFeb13_39 started at: 2024-02-14 04:22:48.760426
Run SEUEquivalenceFeb13_39 ended at: 2024-02-14 04:25:55.231938


2024-02-14 04:27:35,540 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set
2024-02-14 04:29:15,896 - ERROR:Script_Logger:GUI Message of type Error: Unable to read address space 'Waveform Sampler' because the i2c address is not set


## Dump I2C Config Before Run


In [ ]:
i2c_conn.dump_config(config_outdir, "PreRun")

## Do Run

In [ ]:
run_numbers = ["SEUJan24_22"]
total_time = 1
daq_time = 45


for run_number in run_numbers:
    print(f"Run {run_number} started at:", datetime.datetime.now().isoformat(sep=" "))
    run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number=run_number, total_time=total_time, daq_time=daq_time, run_options="--compressed_binary --skip_translation", reset_string="--check_valid_data_start")
    print(f"Run {run_number} ended at:", datetime.datetime.now().isoformat(sep=" "))

## Dump I2C Config After Run


In [ ]:
i2c_conn.dump_config(config_outdir, "PostRun")